In [1]:
include("./src/methods.jl")
using PyPlot, JuMP

┌ Info: Precompiling Clp [e2554f3b-3117-50c0-817c-e040a3ddf72d]
└ @ Base loading.jl:1260
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1260


In [2]:
using MathOptInterface

In [3]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

c_simplex, A_simplex, b_simplex,var_relation = simplex_format(c,A,b)

atol = 1e-8
max_iter = 1000
xzero = []
num_const, num_var = size(A)
isempty(xzero) ?  x = zeros(num_var) : x = xzero
if ~all(A*x - b .<= atol)
    phaseI = true
    c_phaseI,A_phaseI,b_phaseI,
        x_phaseI,bfs,nfs = phaseI_simplex_problem(A_simplex,b_simplex)
    x_simplex = SimplexFromBFS(c_phaseI,A_phaseI,
        b_phaseI,x_phaseI,index_bfs=bfs,index_nfs=nfs)
end
var_relation

Dict{Any,Any} with 3 entries:
  2 => [2, 3]
  3 => [4, 5]
  1 => [1]

In [4]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

4-element Array{Float64,1}:
  0.0
  0.0
  4.0
 -1.0

In [5]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo

Coin0506I Presolve 0 (-4) rows, 0 (-3) columns and 0 (-8) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value -4
Coin0511I After Postsolve, objective -4, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -4 - 0 iterations time 0.002, Presolve 0.00


3-element Array{Float64,1}:
  4.0
 -0.0
 -4.0

In [6]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,var_relation = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_circus = variable_simplex_to_circus(var_relation,x_simplex)
x_simplex = variable_circus_to_simplex(var_relation,x_circus,cs,As,bs)
x_circus

3-element Array{Float64,1}:
  4.0
  0.0
 -4.0

#### Phase 1


In [7]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

# x = [0,0,0,0,0,0,0,4,1]
cs,As,bs,var_relation = simplex_format(c,A,b)
cpi, Api, bpi, xpi,bfs_pi,nfs_pi = phaseI_simplex_problem(As,bs)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0], [-1.0 1.0 … 0.0 0.0; -1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; 1.0 -0.0 … 0.0 1.0], [0.0; 0.0; 4.0; 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 1.0], [10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9])

In [8]:
x_simplex = SimplexFromBFS(cpi,Api,bpi,xpi,index_bfs=bfs_pi,index_nfs=nfs_pi)
x_circus = variable_simplex_to_circus(var_relation,x_simplex)

3-element Array{Float64,1}:
  1.0
  0.0
 -1.0

In [9]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(cpi)[1]])
@constraint(model,Api*x.==bpi) 
@constraint(model,x.>=zeros(size(cpi)[1])) 
@objective(model, Min, cpi'*x)
optimize!(model)
xo = zeros(size(cpi)[1])
for i = 1:size(cpi)[1]
    xo[i] = value(x[i])
end
# x_circus = variable_simplex_to_circus(var_relation,xo)

# termination_status(model)
# display(typeof(MOI.OPTIMAL))

Coin0506I Presolve 0 (-17) rows, 0 (-13) columns and 0 (-33) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00


In [10]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]
# xo = x_circus

pygui(true)
px= [1,4 ,4 ,1 ,1,1,1 ,1,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [-1,-4,0 ,3 ,-1,3,3 ,3,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([xo[1]],[xo[2]],[xo[3]])

libGL error: MESA-LOADER: failed to open iris (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri)
libGL error: failed to load driver: swrast


PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x7f48a450ff90>

### Checking if BFS is Optimal

In [11]:
function check_optimality(c,A,b,x;index_bfs=[],index_nfs=[])
    c = -c
    # Initial setup
    initial_bfs = x
    e  = 10^-7
    if size(index_bfs)[1]>0
        B = index_bfs
        N = index_nfs
    else
        B  = findall(initial_bfs .> 0+e)
        N  = findall(initial_bfs .<= 0+e)
    end
    xn = initial_bfs[N]; xb = initial_bfs[B];

    Ab = A[:,B]; An = A[:,N]; cb = c[B]; cn = c[N]
    p  = inv(Ab)*b
    Q  = -inv(Ab)*An
    r  = (cb'*Q + cn')'
    if all(r.<= 0+e)
        return 1
    else
        return 0
    end
end
check_optimality(cpi,Api,bpi,xo)

1

In [12]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,var_relation = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_circus  = variable_simplex_to_circus(var_relation,x_simplex)
active_ix = [1,2,3]
active_ix_simplex = []
for i in active_ix
    push!(active_ix_simplex,var_relation[i][1])
end
nactive_ix_simplex = filter(x -> !(x in active_ix_simplex),collect(1:size(x_simplex)[1]))

check_optimality(cs,As,bs,x_simplex,index_bfs=active_ix_simplex,index_nfs=nactive_ix_simplex)

1

### Importing MPS file test case 

In [13]:
model = JuMP.read_from_file("../TestCases/afiro.mps")
set_optimizer(model,Clp.Optimizer)
optimize!(model)
variables = JuMP.all_variables(model)
X_sol = zeros(size(variables)[1])
for (i,v) in enumerate(variables)
    X_sol[i] = value(v)
end

@show objective_value(model);
@show X_sol;


list_of_constraint_types(model)

gc = JuMP.all_constraints(model,VariableRef, MOI.GreaterThan{Float64})
ec = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.EqualTo{Float64})
lc = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64})
cj = JuMP.objective_function(model)

variables = JuMP.all_variables(model)
A = zeros(1,size(variables)[1])
b= zeros(1)

c = zeros(1,size(variables)[1])
c[1,indexin(cj.terms.keys,variables)] = cj.terms.vals
c = c'[:]

for i in 1:size(lc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(lc[i]).func.terms
    lc_value = constraint_object(lc[i]).set.upper
    if i == 1
        A[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        b = lc_value
    else
        Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        A = vcat(A,Aux)
        b = vcat(b,lc_value)
    end
end

for i in 1:size(gc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(gc[i]).func
    lc_value = constraint_object(gc[i]).set.lower
    Aux[1,findfirst(variables .== lc_terms)] = -1
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
end

for i in 1:size(ec)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(ec[i]).func.terms
    lc_value = constraint_object(ec[i]).set.value
    Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
    
    Aux[1,indexin(lc_terms.keys,variables)] = -lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,-lc_value)
end


objective_value(model) = -464.75314285714296
X_sol = [339.94285714285724, 0.0, 0.0, 0.0, 0.0, 54.50000000000001, 0.0, 0.0, 19.307142857142853, 0.0, 25.499999999999993, 0.0, 500.00000000000006, 18.214285714285708, 84.80000000000001, 0.0, 0.0, 24.079999999999984, 80.0, 18.214285714285708, -2.3606070955485056e-14, 215.00000000000003, 0.0, -5.684341886080802e-14, 0.0, 0.0, 0.0, 475.92000000000013, 0.0, 0.0, 0.0, 383.94285714285724]
Coin0506I Presolve 6 (-21) rows, 9 (-23) columns and 26 (-57) elements
Clp0006I 0  Obj 0 Dual inf 1,8228541 (3)
Clp0006I 3  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 3 iterations time 0,002, Presolve 0,00


In [43]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo
@show objective_value(model);

objective_value(model) = -464.75314285714285
Coin0506I Presolve 11 (-56) rows, 16 (-16) columns and 58 (-91) elements
Clp0006I 0  Obj 0 Dual inf 2,199997 (3)
Clp0006I 4  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 4 iterations time 0,002, Presolve 0,00


In [45]:
cs,As,bs,var_relation = simplex_format(c,A,b)
model = Model(Clp.Optimizer)
@variable(model, x[1:size(cs)[1]])
@constraint(model,As*x.==bs) 
@constraint(model,x.>=zeros(size(cs)[1])) 
@objective(model, Min, cs'*x)
optimize!(model)
@show objective_value(model);

objective_value(model) = -464.75314285714285
Coin0506I Presolve 13 (-89) rows, 20 (-47) columns and 86 (-133) elements
Clp0006I 0  Obj 0 Primal inf 688 (4) Dual inf 4,217396 (4)
Clp0006I 8  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 8 iterations time 0,002, Presolve 0,00


In [46]:
model

A JuMP Model
Minimization problem with:
Variables: 67
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 35 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 67 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Clp
Names registered in the model: x